In [1]:
import pandas as pd
import numpy as np
import re
import pickle

from pymongo import MongoClient

import sys
sys.path.insert(0, '../src/features/')
sys.path.insert(0, '../src/data/')
%load_ext autoreload

%autoreload 1
%aimport spacy_lemmer
import spacy_lemmer

In [2]:
from clean_book import book_cleaner


In [3]:
client=MongoClient()
db = client['proj_4_gutenberg']
collection = db['gutenberg_books']

book_ids = collection.find({}, {'_id':1, 'title':1, 'author':1})
book_nums = []
book_titles = []
book_authors = []
for book in book_ids:
    #Imp of Earn (15399) inputs many proper nouns, Moby Dick (2148) and War and Peace(2600) and Jane Eyre (1260) are too long, 
    #and is breaking the lemmatizer, so for now, I am removing them.
#     if book['_id'] != '2148' and book['_id']!='15399' and book['_id']!='2600' and book['_id']!='1260':
#         book_nums.append((book['_id']))
    book_nums.append((book['_id']))
    book_titles.append((book['title']))
    book_authors.append((book['author']))

In [263]:
# books_for_lemming = [book_cleaner(str(book)) for book in book_nums]
# spacy_lemmed_text = [spacy_lemmer.spacy_lem_text(book) for book in books_for_lemming]

Pride and Prejudice 499000
Frankenstein 415792
Heart of Darkness 208333
A Modest Proposal 19173
A Tale of Two Cities 499000
The Importance of Being Earnest 116522
Alice's Adventures in Wonderland 133809
Moby Dick; or The Whale 499000
Voyage to Jupiter 389891
The Adventures of Sherlock Holmes 499000
The Souls of Black Folk 395540
Metamorphosis 117359
The Yellow Wallpaper 30931
The Earth Quarter 109426
War and Peace 499000
Grimms' Fairy Tales 499000
Dracula 499000
Jane Eyre 499000
The Picture of Dorian Gray 424422
Walden, and On The Duty Of Civil Disobedience 499000
The Count of Monte Cristo 499000
Leaves of Grass 8758
Autographs for Freedom 295429
Great Expectations 499000
Peter Pan 250581
Dubliners 364177
The Narrative of the Life of Frederick Douglass 217104
Pygmalion 184851
The Romance of Lust 499000
Ulysses 499000
Siddhartha 213768
A Christmas Carol 155526
Treasure Island 355349
Leviathan 499000
Legendary Tales of the Highlands (Volume 1 of 3) 312155
Wuthering Heights 499000
The Ess

In [315]:
# with open('../data/processed/spacy_lemmed_train_text_500000.pickle', "wb") as output_file:
#     pickle.dump(lemmed_text, output_file)

In [7]:
!ls ../data/

external  interim   processed raw


In [ ]:
# with open('../data/processed/lemmed_train_data.pickle', "wb") as output_file:
#     pickle.dump(lemmed_text, output_file)

In [6]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import scale
from sklearn.cluster import KMeans

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
with open('../data/processed/spacy_lemmed_train_text_500000.pickle', "rb") as input_file:
    lemmed_text = pickle.load(input_file)


FileNotFoundError: [Errno 2] No such file or directory: '../data/processed/spacy_lemmed_train_text_500000.pickle'

In [373]:
book_titles[50]

'Candide'

In [374]:
lemmed_text[50]

' sound . a modern writer would have try to catch and fix in word some of those atlantic change which break the monotony of that voyage from to . when and be sail the length of the should have have a contrast between naked scarp balearic cliff and headland of in mist . should have have quarter distance , far horizon , the alter silhouette of an ionian island . colored bird would have fill with silver or acid cry . . pangloss , to prove the existence of design in the universe , say \n [ viii ] \n that nose be make to carry spectacle , and so have spectacle . a modern satirist would not try to paint with \'s quick brush the doctrine that want to expose . and would choose a more complicated doctrine than \'s optimism , would study more closely , feel destructive way about with a more learned and caress malice . attack , stealthier , more flexible and more patient than \'s , would call upon , especially when learning get a little out of control , to be more than patient . now and then woul

In [375]:
tfidf= TfidfVectorizer(min_df = .1, max_df = .9, stop_words='english')
doc_word = tfidf.fit_transform(lemmed_text)

## Truncated SVD

In [405]:
lsa = TruncatedSVD(50)
doc_topic = lsa.fit_transform(doc_word)
lsa.explained_variance_ratio_

array([0.03456249, 0.05922613, 0.05126619, 0.03408698, 0.03049677,
       0.02721064, 0.02527387, 0.02489807, 0.02447538, 0.02192053,
       0.0213375 , 0.01952326, 0.0193305 , 0.0187318 , 0.01825443,
       0.0178374 , 0.01752647, 0.01716408, 0.01685279, 0.01657619,
       0.01628603, 0.0158388 , 0.0155814 , 0.01509097, 0.01495459,
       0.01439036, 0.01374566, 0.01348573, 0.01313087, 0.01282269,
       0.01234683, 0.01232019, 0.01225002, 0.01188097, 0.01155685,
       0.01132801, 0.01086247, 0.01073153, 0.01047309, 0.01036333,
       0.01002436, 0.00993622, 0.00982744, 0.00958496, 0.00940638,
       0.00930961, 0.00906777, 0.00891102, 0.00886352, 0.00868456])

In [406]:
sum(lsa.explained_variance_ratio_)

0.8696077216324952

In [407]:
!ls

#toc_strip.py#                      new_text_predictor.ipynb
1_train_test_tfidf.ipynb            toc_strip.py~
2_train_with_imported_modules.ipynb vector_all_books.ipynb
3_kmeans_with_classes.ipynb         vector_multi_books.ipynb
Gut_scrape.ipynb                    vector_three_books.ipynb
Mongo_insert_onebook_function.ipynb vectorize.ipynb
external_book_classification.ipynb  vectorize.py
external_book_with_modules.ipynb    vectorize_three.py


In [408]:
# with open('../src/models/tfidf_73.pickle', "wb") as output_file:
#     pickle.dump(tfidf, output_file)

In [409]:
# with open('../src/models/lsa_73.pickle', "wb") as output_file:
#     pickle.dump(lsa, output_file)

In [410]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

display_topics(lsa, tfidf.get_feature_names(), 10)


Topic  0
father, oh, dear, yes, mother, boy, girl, sir, lady, sister

Topic  1
opinion, subject, sister, hath, manner, nature, government, pleasure, law, general

Topic  2
sister, dear, oh, girl, lady, mother, yes, marry, father, gentleman

Topic  3
thou, thee, thy, doth, ship, unto, reply, sea, hath, father

Topic  4
ship, captain, sea, boat, island, shore, sail, voyage, reply, majesty

Topic  5
slave, slavery, boy, mother, master, ship, pirate, dollar, negro, nigger

Topic  6
father, sister, mother, gentleman, street, thee, thou, car, job, thy

Topic  7
hath, boy, girl, sect, kingdom, king, government, gentleman, sir, wealth

Topic  8
forest, father, king, town, wife, mother, princess, food, river, teaching

Topic  9
tenement, dollar, planet, sister, satellite, mother, job, girl, car, ship

Topic  10
tone, minute, sister, remark, wood, hath, kitten, jury, slave, reply

Topic  11
sect, pattern, sister, government, boy, law, window, father, paper, legislative

Topic  12
ship, ages, st

In [411]:
book_titles

['Pride and Prejudice',
 'Frankenstein',
 'Heart of Darkness',
 'A Modest Proposal',
 'A Tale of Two Cities',
 'The Importance of Being Earnest',
 "Alice's Adventures in Wonderland",
 'Moby Dick; or The Whale',
 'Voyage to Jupiter',
 'The Adventures of Sherlock Holmes',
 'The Souls of Black Folk',
 'Metamorphosis',
 'The Yellow Wallpaper',
 'The Earth Quarter',
 'War and Peace',
 "Grimms' Fairy Tales",
 'Dracula',
 'Jane Eyre',
 'The Picture of Dorian Gray',
 'Walden, and On The Duty Of Civil Disobedience',
 'The Count of Monte Cristo',
 'Leaves of Grass',
 'Autographs for Freedom',
 'Great Expectations',
 'Peter Pan',
 'Dubliners',
 'The Narrative of the Life of Frederick Douglass',
 'Pygmalion',
 'The Romance of Lust',
 'Ulysses',
 'Siddhartha',
 'A Christmas Carol',
 'Treasure Island',
 'Leviathan',
 'Legendary Tales of the Highlands (Volume 1 of 3)',
 'Wuthering Heights',
 'The Essays of Montaigne, Complete',
 'Les Misérables',
 'Songs of Innocence and Songs of Experience',
 'A Stu

In [333]:
Vt = pd.DataFrame(doc_topic.round(5),
             index = book_titles,
             columns = ["component_1","component_2", 'component_3', 'component_4', 'component_5','component_6','component_7'])
# Vt.sort_values('component_5', ascending=False)
Vt

ValueError: Shape of passed values is (20, 72), indices imply (7, 72)

## Test Book 

In [412]:
# with open('../src/models/tfidf_73.pickle', "rb") as input_file:
#     tfidf = pickle.load(input_file)
    
# with open('../src/models/lsa_73.pickle', "rb") as input_file:
#     lsa = pickle.load(input_file)

In [413]:
file_loc = '/Users/mbarry/Documents/Coding/ds/metis/projects/project_04/proj_4/data/external/becoming.txt'
title_of_test = 'becoming'

In [414]:
import book_to_compare
lemmed_test = book_to_compare.new_test_book(file_loc, title_of_test)
print(lemmed_test)

5971
[' sound a little like a bad joke , do not ? what happen when a solitude - love individualist marrie an outgoing family woman who do not love solitude one bit?the answer , be guess , be probably the good and most sustaining answer to nearly every question arise inside a marriage , no matter who be or what the issue be : find way to adapt . if be in for ever , there ’ really no choice . ’s graduation photofacebook   ‘ have so much – an education , a healthy sense of self , a deep arsenal of ambition – and be wise enough to credit mother with instill in : of the - be to say that at the start of   , fly to and spend about five week live alone with thought while work on a draft of book dreams from , fill yellow legal pad with fastidious handwriting , distil idea during languid daily walk amid the coconut palm and lapping tide . , meanwhile , stay home on , live upstairs from mother , , as another leaden winter descend , shellac the tree and sidewalk with ice . keep busy , see friend a

In [415]:
test_tfidf = tfidf.transform(lemmed_test)
test_lsa = lsa.transform(test_tfidf)


In [416]:
client=MongoClient()
db = client['proj_4_gutenberg']
collection = db['gutenberg_books']

book_ids = collection.find({}, {'_id':1, 'title':1, 'author':1})
book_nums = []
book_titles = []
book_authors= []
for book in book_ids:
    #Imp of Earn (15399) inputs many proper nouns, Moby Dick (2148) and War and Peace(2600) and Jane Eyre (1260) are too long, 
    #and is breaking the lemmatizer, so for now, I am removing them.
#     if book['_id'] != '2148' and book['_id']!='15399' and book['_id']!='2600' and book['_id']!='1260':
#         book_nums.append((book['_id']))
    book_nums.append((book['_id']))
    book_titles.append((book['title']))
    book_authors.append((book['author']))

In [417]:

book_titles.append(book_to_compare.test_title(title_of_test))


In [418]:
sorted(book_titles)

['A Christmas Carol',
 'A Modest Proposal',
 'A Study In Scarlet',
 'A Tale of Two Cities',
 "Alice's Adventures in Wonderland",
 'An Occurrence at Owl Creek',
 'Anne of Green Gables',
 'Anthem',
 'Around the World in 80 Days',
 'Autobiography of Benjamin Franklin',
 'Autographs for Freedom',
 'Beyond Good and Evil',
 'Candide',
 'Dracula',
 'Dubliners',
 'Emma',
 'Frankenstein',
 'Great Expectations',
 "Grimms' Fairy Tales",
 "Gulliver's Travels",
 'Hard Times',
 'Heart of Darkness',
 'How the Other Half Lives',
 'Jane Eyre',
 'Journey Work',
 'Leaves of Grass',
 'Legendary Tales of the Highlands (Volume 1 of 3)',
 'Les Misérables',
 'Leviathan',
 'Little Women',
 'Mansfield Park',
 'Meditations',
 'Metamorphosis',
 'Moby Dick; or The Whale',
 'My Secret Life, Volumes I. to III.',
 'Northanger Abbey',
 'Oliver Twist',
 'On Liberty',
 'Peter Pan',
 'Pride and Prejudice',
 'Pygmalion',
 'Second Treatise of Government',
 'Sense and Sensibility',
 'Siddhartha',
 'Songs of Innocence and So

In [390]:
sorted(book_authors)

['A. Conan Doyle',
 'Alexandre Dumas, père',
 'Ambrose Bierce',
 'Anonymous',
 'Anonymous',
 'Arthur Conan Doyle',
 'Arthur Conan Doyle',
 'Ayn Rand',
 'Benjamin Franklin',
 'Bram Stoker',
 'Charles Dickens',
 'Charles Dickens',
 'Charles Dickens',
 'Charles Dickens',
 'Charles Dickens',
 'Charles Dodgson, AKA Lewis Carroll',
 'Charlotte Bronte',
 'Charlotte Perkins Gilman',
 'Damon Knight',
 'Daniel Defoe',
 'Dave Dryfoos',
 'David Morrison',
 'Emily Bronte',
 'Franz Kafka',
 'Frederick Douglass',
 'Friedrich Nietzsche',
 'Fyodor Dostoyevsky',
 'George Bernard Shaw',
 'H. G. Wells',
 'Harriet Beecher Stowe',
 'Henry David Thoreau',
 'Herman Hesse',
 'Herman Melville',
 'Jacob A. Riis',
 'James Joyce',
 'James Joyce',
 'James M. Barrie',
 'Jane Austen',
 'Jane Austen',
 'Jane Austen',
 'Jane Austen',
 'Jane Austen',
 'John Locke',
 'John Stuart Mill',
 'Jonathan Swift',
 'Jonathan Swift',
 'Joseph Conrad',
 'Jules Verne',
 'Leo Tolstoy',
 'Lewis Carroll',
 'Louisa May Alcott',
 'Lucy M

In [419]:
from sklearn.preprocessing import scale
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 


doc_for_k = np.concatenate([doc_topic, test_lsa], axis=0)
doc_for_k

array([[ 6.62687940e-01,  4.35673466e-01,  4.10375776e-01, ...,
        -2.60524709e-02,  2.44971187e-03,  9.50543402e-03],
       [ 6.21966309e-01,  2.33366676e-01, -8.40499015e-02, ...,
        -1.15237961e-01, -3.31347723e-02, -1.42875391e-02],
       [ 5.10492146e-01, -2.32446438e-01, -1.87746028e-01, ...,
        -1.06099098e-01, -6.35921360e-02,  4.25311461e-04],
       ...,
       [ 4.67340126e-01, -6.04323204e-02, -1.70652874e-01, ...,
         8.55466131e-02,  7.09131003e-02, -7.30480056e-02],
       [ 5.84035262e-01, -2.70573808e-01, -1.61148993e-02, ...,
         4.88921734e-02,  8.58272929e-02, -1.60366326e-01],
       [ 2.06170523e-01, -1.23256302e-02,  1.29651835e-02, ...,
         5.66636276e-03, -1.39382494e-02,  2.49257304e-02]])

In [420]:
a= cosine_similarity(doc_for_k)
b= np.triu_indices(a.shape[0],a.shape[0])
# np.triu_indices(cosine_similarity(doc_for_k))
a[b]=0
a

array([[1.        , 0.57711106, 0.22600723, ..., 0.27231773, 0.28301019,
        0.51873222],
       [0.57711106, 1.        , 0.3522113 , ..., 0.28480188, 0.35314039,
        0.39734637],
       [0.22600723, 0.3522113 , 1.        , ..., 0.32661899, 0.4096265 ,
        0.31679721],
       ...,
       [0.27231773, 0.28480188, 0.32661899, ..., 1.        , 0.24611926,
        0.23577332],
       [0.28301019, 0.35314039, 0.4096265 , ..., 0.24611926, 1.        ,
        0.39887349],
       [0.51873222, 0.39734637, 0.31679721, ..., 0.23577332, 0.39887349,
        1.        ]])

In [421]:
high_cos_sim = 0
high_title = 'hello'
next_highest_cos = 0
next_high_title = ''
third_highest_cos = 0
third_highest_title = ''
for title, doc in zip(book_titles,doc_for_k):
    book_cos_sim = cosine_similarity((doc, doc_for_k[-1]))[1][0]
    if title != title_of_test and book_cos_sim > high_cos_sim:
        third_highest_cos = next_highest_cos
        third_highest_title = next_high_title
        next_highest_cos = high_cos_sim
        next_high_title = high_title
        high_cos_sim = book_cos_sim
        high_title=title


print(high_title, high_cos_sim)
print(next_high_title, next_highest_cos)
print(third_highest_title, third_highest_cos)

Little Women 0.7104693744989053
Uncle Tom's Cabin 0.6241236146338112
Dubliners 0.6093092725058453


In [422]:
num_clusters = 15
km = KMeans(n_clusters=num_clusters,random_state=10,n_init=1) # n_init, number of times the K-mean algorithm will run
km.fit(doc_for_k)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=15, n_init=1, n_jobs=None, precompute_distances='auto',
    random_state=10, tol=0.0001, verbose=0)

In [423]:
group_labels = km.labels_
for group,title in zip(group_labels, book_titles):
    if group == 2:
        print(title, group)

Moby Dick; or The Whale 2
Treasure Island 2
The Life and Adventures of Robinson Crusoe 2


In [424]:
group_labels = km.labels_
test_group = ''
for group,title in zip(group_labels, book_titles):
    if title == title_of_test:
        test_group = group
for group,title in zip(group_labels, book_titles):
    if group == test_group:
        print(title, group)

Metamorphosis 6
The Yellow Wallpaper 6
War and Peace 6
Grimms' Fairy Tales 6
Peter Pan 6
Pygmalion 6
The Romance of Lust 6
Uncle Tom's Cabin 6
Little Women 6
Anne of Green Gables 6
The Adventures of Tom Sawyer, Complete 6
becoming 6


## NMF

In [425]:
from sklearn.decomposition import NMF


In [454]:
nmf_model = NMF(20)
doc_topic = nmf_model.fit_transform(doc_word)
doc_topic

array([[0.        , 0.58736435, 0.        , ..., 0.00688944, 0.        ,
        0.        ],
       [0.00963262, 0.22327378, 0.07488593, ..., 0.        , 0.        ,
        0.01837453],
       [0.01048006, 0.        , 0.07745442, ..., 0.09112715, 0.        ,
        0.02704292],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.14065244, 0.01237476, 0.1091555 , ..., 0.05694098, 0.        ,
        0.        ],
       [0.00563525, 0.        , 0.06827234, ..., 0.00657702, 0.06034158,
        0.        ]])

In [455]:
display_topics(nmf_model, tfidf.get_feature_names(), 10)


Topic  0
sir, gentleman, yes, reply, moor, father, window, lady, street, fellow

Topic  1
sister, father, dear, mother, lady, manner, feeling, pleasure, general, oh

Topic  2
ship, sea, boat, captain, island, shore, sail, voyage, squire, ye

Topic  3
thou, thee, thy, unto, doth, hath, thyself, aye, thine, weep

Topic  4
white, earth, sky, black, strange, forest, sun, machine, wire, darkness

Topic  5
slave, slavery, master, nigger, white, whip, negro, plantation, black, freedom

Topic  6
boy, girl, oh, mother, lady, yes, school, pirate, laugh, mrs

Topic  7
hath, government, sect, law, subject, wealth, nature, opinion, legislative, whereof

Topic  8
father, town, mother, wife, forest, king, food, reply, princess, son

Topic  9
philosopher, morality, opinion, soul, moral, truth, philosophy, german, self, individual

Topic  10
tone, oh, minute, remark, dear, reply, jury, kitten, mouse, horse

Topic  11
train, car, job, bail, dollar, major, deputy, machine, truck, ad

Topic  12
rack, pla

In [456]:
num_clusters = 10
km = KMeans(n_clusters=num_clusters,random_state=10,n_init=1) # n_init, number of times the K-mean algorithm will run
km.fit(doc_topic)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=10, n_init=1, n_jobs=None, precompute_distances='auto',
    random_state=10, tol=0.0001, verbose=0)

In [470]:
group_labels = km.labels_
for group,title in zip(group_labels, book_titles):
    if group == 0:
        print(title, group)

A Tale of Two Cities 0
The Adventures of Sherlock Holmes 0
The Earth Quarter 0
Dracula 0
Jane Eyre 0
The Picture of Dorian Gray 0
The Count of Monte Cristo 0
Great Expectations 0
Dubliners 0
Ulysses 0
A Christmas Carol 0
Wuthering Heights 0
A Study In Scarlet 0
Uncle Tom's Cabin 0
Hard Times 0
The Hound of the Baskervilles 0
Oliver Twist 0
Around the World in 80 Days 0


## LDA

In [353]:
# gensim
from gensim import corpora, models, similarities, matutils

# sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer

# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [354]:
count_vectorizer = CountVectorizer(ngram_range=(1, 2),  
                                   stop_words='english')

doc_word_for_gen = count_vectorizer.fit_transform(lemmed_text).transpose()



In [355]:
doc_word_for_gen.shape

(1108201, 72)

In [356]:
doc_word_for_gen.shape

(1108201, 72)

In [358]:
import pandas as pd

pd.DataFrame(doc_word_for_gen.toarray(), count_vectorizer.get_feature_names())[300:400]

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
abate grow,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
abate hot,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abate hour,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abate jot,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abate little,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abate lust,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abate meantime,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abate merriment,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abate oftener,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abate old,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [359]:
corpus = matutils.Sparse2Corpus(doc_word_for_gen)

In [360]:
id2word = dict((v, k) for k, v in count_vectorizer.vocabulary_.items())

In [361]:
len(id2word)

1108201

In [364]:
lda = models.LdaModel(corpus=corpus, num_topics=7, id2word=id2word, passes=5)

2019-02-26 16:47:06,649 : INFO : using symmetric alpha at 0.14285714285714285
2019-02-26 16:47:06,650 : INFO : using symmetric eta at 0.14285714285714285
2019-02-26 16:47:06,821 : INFO : using serial LDA version on this node
2019-02-26 16:47:07,545 : INFO : running online (multi-pass) LDA training, 7 topics, 5 passes over the supplied corpus of 72 documents, updating model once every 72 documents, evaluating perplexity every 72 documents, iterating 50x with a convergence threshold of 0.001000
2019-02-26 16:47:07,546 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2019-02-26 16:47:22,480 : INFO : -16.498 per-word bound, 92560.5 perplexity estimate based on a held-out corpus of 72 documents with 3268818 words
2019-02-26 16:47:22,481 : INFO : PROGRESS: pass 0, at document #72/72
2019-02-26 16:47:25,236 : INFO : topic #6 (0.143): 0.006*"say" + 0.004*"come" + 0.003*"man" + 0.003*"know" + 0.003*"make" + 0.00

In [365]:
lda.print_topics()

2019-02-26 16:49:44,414 : INFO : topic #0 (0.143): 0.008*"say" + 0.004*"man" + 0.004*"come" + 0.003*"make" + 0.003*"know" + 0.003*"good" + 0.003*"time" + 0.003*"think" + 0.002*"look" + 0.002*"little"
2019-02-26 16:49:44,431 : INFO : topic #1 (0.143): 0.008*"say" + 0.003*"man" + 0.003*"come" + 0.003*"make" + 0.003*"know" + 0.003*"think" + 0.002*"look" + 0.002*"time" + 0.002*"good" + 0.002*"like"
2019-02-26 16:49:44,445 : INFO : topic #2 (0.143): 0.007*"say" + 0.004*"come" + 0.003*"make" + 0.002*"man" + 0.002*"time" + 0.002*"knight" + 0.002*"day" + 0.002*"good" + 0.002*"know" + 0.002*"great"
2019-02-26 16:49:44,460 : INFO : topic #3 (0.143): 0.003*"man" + 0.002*"say" + 0.002*"make" + 0.002*"power" + 0.001*"good" + 0.001*"law" + 0.001*"people" + 0.001*"right" + 0.001*"time" + 0.001*"little"
2019-02-26 16:49:44,494 : INFO : topic #4 (0.143): 0.005*"man" + 0.003*"say" + 0.002*"make" + 0.002*"thing" + 0.002*"good" + 0.002*"come" + 0.002*"time" + 0.002*"know" + 0.002*"think" + 0.002*"like"
20

[(0,
  '0.008*"say" + 0.004*"man" + 0.004*"come" + 0.003*"make" + 0.003*"know" + 0.003*"good" + 0.003*"time" + 0.003*"think" + 0.002*"look" + 0.002*"little"'),
 (1,
  '0.008*"say" + 0.003*"man" + 0.003*"come" + 0.003*"make" + 0.003*"know" + 0.003*"think" + 0.002*"look" + 0.002*"time" + 0.002*"good" + 0.002*"like"'),
 (2,
  '0.007*"say" + 0.004*"come" + 0.003*"make" + 0.002*"man" + 0.002*"time" + 0.002*"knight" + 0.002*"day" + 0.002*"good" + 0.002*"know" + 0.002*"great"'),
 (3,
  '0.003*"man" + 0.002*"say" + 0.002*"make" + 0.002*"power" + 0.001*"good" + 0.001*"law" + 0.001*"people" + 0.001*"right" + 0.001*"time" + 0.001*"little"'),
 (4,
  '0.005*"man" + 0.003*"say" + 0.002*"make" + 0.002*"thing" + 0.002*"good" + 0.002*"come" + 0.002*"time" + 0.002*"know" + 0.002*"think" + 0.002*"like"'),
 (5,
  '0.001*"say" + 0.001*"man" + 0.001*"know" + 0.000*"time" + 0.000*"make" + 0.000*"good" + 0.000*"look" + 0.000*"think" + 0.000*"come" + 0.000*"like"'),
 (6,
  '0.004*"say" + 0.003*"come" + 0.003*"